## Location for opening  a new Indian restaurant 5 kilo meters from New york city CBD.

####    Introduction/Business Problem

In [37]:
# An enthusiast wants to open an Indian restaurant within 5 kms from New York CBD. 
# Being a new entrepreneur in this domain , he/she wants a visualization map of Indian restaurant 's locations within this area.
# And he wants a suggestive map location  to establish the business,where there are non or few Indian restaurant's located.

####    Data section 

In [38]:
# Data Used: The latitude and longitude of New York city  is used to make a call to the 4 square API  with search query  as Indian restaurant and within a radius of 5 Km's.
# For example: The search query for New York city for Indian restaurant's within a radius of 5 Kms from CBD with a search  limit of 5000 restaurants will be:
# 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


#### Import necessary Libraries


In [39]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

In [40]:
CLIENT_ID = '22CNRY0G2PZV00WLW12BFAC5KG0ZPB1CDZ2HQ05HNTO203UL' # your Foursquare ID
CLIENT_SECRET = 'LRV2TWX04RQ4WTTGSVLJS4MBTVW5GRUOK4ASBAAOH1JPGHEN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 3000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22CNRY0G2PZV00WLW12BFAC5KG0ZPB1CDZ2HQ05HNTO203UL
CLIENT_SECRET:LRV2TWX04RQ4WTTGSVLJS4MBTVW5GRUOK4ASBAAOH1JPGHEN


#### Get the latitude and longitiude of New York

In [41]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Search four sqaure API using serach query as Indian restaurant,radius=5000 meters and a limit of 5000 restuarants

In [42]:
search_query = 'Indian restaurant'
radius = 5000
print(search_query + ' .... OK!')

Indian restaurant .... OK!


In [43]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=22CNRY0G2PZV00WLW12BFAC5KG0ZPB1CDZ2HQ05HNTO203UL&client_secret=LRV2TWX04RQ4WTTGSVLJS4MBTVW5GRUOK4ASBAAOH1JPGHEN&ll=40.7127281,-74.0060152&v=20180604&query=Indian restaurant&radius=5000&limit=3000'

In [44]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee9affa7a98810d7eb94137'},
 'response': {'venues': [{'id': '4116be80f964a520f90b1fe3',
    'name': 'Panna II Garden Indian Restaurant',
    'location': {'address': '93 1st Ave',
     'crossStreet': 'btwn 5th & 6th St',
     'lat': 40.726272990045985,
     'lng': -73.98627301759028,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.726272990045985,
       'lng': -73.98627301759028},
      {'label': 'entrance', 'lat': 40.726254, 'lng': -73.986488}],
     'distance': 2246,
     'postalCode': '10003',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['93 1st Ave (btwn 5th & 6th St)',
      'New York, NY 10003',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [45]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood
0,4116be80f964a520f90b1fe3,Panna II Garden Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1592373225,False,93 1st Ave,btwn 5th & 6th St,40.726273,-73.986273,"[{'label': 'display', 'lat': 40.72627299004598...",...,United States,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...",149505937,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4d0e6159bf0c8eec2188282a,Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1592373225,False,369 Broome St,NaN,40.719978,-73.995903,"[{'label': 'display', 'lat': 40.71997833251953...",...,United States,"[369 Broome St, New York, NY 10013, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,506a2625498e3c434686abbc,Asya Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1592373225,False,46 Henry St,NaN,40.699607,-73.992110,"[{'label': 'display', 'lat': 40.699607, 'lng':...",...,United States,"[46 Henry St, Brooklyn, NY 11201, United States]",94617937,266126,https://www.seamless.com/menu/asya-46-henry-st...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
3,4c17b29b838020a1fd0de361,Bombay's Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1592373225,False,60 Pearl St,Broad St.,40.703569,-74.011141,"[{'label': 'display', 'lat': 40.70356930142193...",...,United States,"[60 Pearl St (Broad St.), New York, NY 10004, ...",93243558,263979,https://www.seamless.com/menu/bombays-indian-r...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
4,4f32a9c119836c91c7eda319,Indian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1592373225,False,338 9th Ave,NaN,40.750366,-73.998138,"[{'label': 'display', 'lat': 40.7503662109375,...",...,United States,"[338 9th Ave, New York, NY 10001, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Define information of interest and filter dataframe

In [46]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Panna II Garden Indian Restaurant,Indian Restaurant,93 1st Ave,btwn 5th & 6th St,40.726273,-73.986273,"[{'label': 'display', 'lat': 40.72627299004598...",2246,10003,US,New York,NY,United States,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...",NaN,4116be80f964a520f90b1fe3
1,Indian Restaurant,Indian Restaurant,369 Broome St,NaN,40.719978,-73.995903,"[{'label': 'display', 'lat': 40.71997833251953...",1174,10013,US,New York,NY,United States,"[369 Broome St, New York, NY 10013, United Sta...",NaN,4d0e6159bf0c8eec2188282a
2,Asya Indian Restaurant,Indian Restaurant,46 Henry St,NaN,40.699607,-73.992110,"[{'label': 'display', 'lat': 40.699607, 'lng':...",1873,11201,US,Brooklyn,NY,United States,"[46 Henry St, Brooklyn, NY 11201, United States]",NaN,506a2625498e3c434686abbc
3,Bombay's Indian Restaurant,Indian Restaurant,60 Pearl St,Broad St.,40.703569,-74.011141,"[{'label': 'display', 'lat': 40.70356930142193...",1107,10004,US,New York,NY,United States,"[60 Pearl St (Broad St.), New York, NY 10004, ...",NaN,4c17b29b838020a1fd0de361
4,Indian Restaurant,Food,338 9th Ave,NaN,40.750366,-73.998138,"[{'label': 'display', 'lat': 40.7503662109375,...",4242,10001,US,New York,NY,United States,"[338 9th Ave, New York, NY 10001, United States]",NaN,4f32a9c119836c91c7eda319
5,Dhaka Indian Restaurant,Indian Restaurant,148 Atlantic Ave,NaN,40.690147,-73.994926,"[{'label': 'display', 'lat': 40.69014739990234...",2682,11201,US,Brooklyn,NY,United States,"[148 Atlantic Ave, Brooklyn, NY 11201, United ...",NaN,4e4de5fbbd4101d0d79da9d0
6,Brick Lane Exotic Indian Restaurant,Indian Restaurant,142 Smith St,Bergen,40.687031,-73.990509,"[{'label': 'entrance', 'lat': 40.686988, 'lng'...",3145,11201,US,Brooklyn,NY,United States,"[142 Smith St (Bergen), Brooklyn, NY 11201, Un...",NaN,4fe9fae4e4b03a10ef9d02d2
7,Kanan's Indian Restaurant,Indian Restaurant,452 3rd Ave,9th Street,40.671891,-73.990940,"[{'label': 'display', 'lat': 40.67189099999999...",4720,11215,US,Brooklyn,NY,United States,"[452 3rd Ave (9th Street), Brooklyn, NY 11215,...",NaN,52f18573498ec2c34e830ffd
8,Joy Indian Restaurant,Indian Restaurant,301 Flatbush Ave,Prospect Pl,40.678788,-73.973581,"[{'label': 'display', 'lat': 40.67878769589125...",4665,11217,US,Brooklyn,NY,United States,"[301 Flatbush Ave (Prospect Pl), Brooklyn, NY ...",NaN,4a7a2112f964a5209ee81fe3
9,Sapid Indian Restaurant,Indian Restaurant,595 Washington Ave,btw Pacific & Dean,40.679789,-73.964170,"[{'label': 'display', 'lat': 40.67978864648759...",5091,11238,US,Brooklyn,NY,United States,"[595 Washington Ave (btw Pacific & Dean), Broo...",NaN,4c673e729cb82d7f024593d2


<h3> List of Indian Restaurants located within  5 km radius of new york city<h3>

In [47]:
dataframe_filtered.name

0                     Panna II Garden Indian Restaurant
1                                     Indian Restaurant
2                                Asya Indian Restaurant
3                            Bombay's Indian Restaurant
4                                     Indian Restaurant
5                               Dhaka Indian Restaurant
6                   Brick Lane Exotic Indian Restaurant
7                             Kanan's Indian Restaurant
8                                 Joy Indian Restaurant
9                               Sapid Indian Restaurant
10    Vatan Indian Restaurant & Caterers in Jersey C...
11                Joy Curry & Tandoor Indian Restaurant
12                                New Shezan Restaurant
13                       Indian Biryani Delights (Cart)
14                                       Indian Express
15                         TJ Byrnes Bar and Restaurant
16            Indian Beautiful Art (IBA) Crafts PVT Ltd
17                        Bismilla Indian Biryan

In [48]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel



# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map